In [37]:
import pandas as pd
import numpy as np
from time import time
from nltk.tokenize import sent_tokenize, word_tokenize 
from string import punctuation
from nltk.corpus import stopwords
import nltk
from time import time

In [14]:
df = pd.read_csv('final_english_reviews.csv')

In [15]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'user_id', 'review_id', 'text',
       'business_id', 'stars', 'date'],
      dtype='object')

In [16]:
df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'],axis=0,inplace=True)

In [17]:
df['stars'].value_counts()

4    69302
5    59386
3    40021
2    22956
1    22889
Name: stars, dtype: int64

In [18]:
df_5 = df[df['stars'] == 5.0]
df_4 = df[df['stars'] == 4.0]
df_3 = df[df['stars'] == 3.0]
df_2 = df[df['stars'] == 2.0]
df_1 = df[df['stars'] == 1.0]

In [19]:
df_5 = df_5.sample(10000)
df_4 = df_4.sample(10000)
df_3 = df_3.sample(10000)
df_2 = df_2.sample(10000)
df_1 = df_1.sample(10000)

In [20]:
df_sampled = pd.concat([df_5,df_4,df_3,df_2,df_1],axis = 0)

In [23]:
df_sampled['stars'].value_counts()

5    10000
4    10000
3    10000
2    10000
1    10000
Name: stars, dtype: int64

In [24]:
docs = df_sampled['text']
len(docs)

50000

In [71]:
docs = list(docs)

In [72]:
docs_tokens = [word_tokenize(doc) for doc in docs]

In [116]:
df_sampled.to_csv('final_sampled_english_reviews.csv')

In [84]:
def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t for t in d if len(t) >= minlen ] for d in docs ]

def remove_stop_words(docs):
    en_stops = stopwords.words('english')
    en_stops.extend(['should','they','this','came','would','could'])
    new_docs = []
    for doc in docs:
        new_word = []  
        for word in doc:
            if word not in en_stops:
                new_word.append(word)
        new_docs.append(new_word)
            
    return new_docs

def filterInput(documents):
    new_docs = []
    for doc in documents:
        new_word = []
        for word in doc:
            new_word.append(word.lower())
            for char in word:
                if(not char.isalpha()):
                    new_word.remove(word.lower())
                    break
        new_docs.append(new_word)
    
    return new_docs

import re
def remove_punctuation(docs):
    new_docs = []
    for doc in docs:
        new_words = []  
        for word in doc:
            new_word = re.sub(r'[^\w\s]', '', word)
            if new_word != '':
                new_words.append(new_word)
        new_docs.append(new_words)
            
    return new_docs

In [85]:
docs_filtered = filterLen(docs_tokens,4)

In [93]:
docs_wo_stopwords = remove_stop_words(docs_filtered)

In [95]:
docs_filt = filterInput(docs_wo_stopwords)

In [99]:
docs_wo_punctuation = remove_punctuation(docs_filt)

In [33]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


# spacy for lemmatization


# # Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [34]:
#Create Corpus
texts = docs_wo_punctuation

# Create Dictionary
id2word = corpora.Dictionary(docs_wo_punctuation)

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [117]:
# Build LDA model
start = time()
lda_model = gensim.models.LdaMulticore(workers=3,corpus=corpus,id2word=id2word, num_topics=20, random_state=100,passes=5)
end = time()
# Print the Keyword in the 15 topics
pprint(lda_model.print_topics())
print((end - start)/60)

[(0,
  '0.024*"breakfast" + 0.023*"eggs" + 0.016*"brunch" + 0.016*"place" + '
  '0.016*"good" + 0.015*"food" + 0.010*"like" + 0.010*"service" + '
  '0.008*"bacon" + 0.007*"vegetarian"'),
 (1,
  '0.013*"good" + 0.013*"dessert" + 0.009*"great" + 0.009*"food" + '
  '0.008*"restaurant" + 0.007*"service" + 0.007*"delicious" + 0.007*"also" + '
  '0.007*"menu" + 0.006*"ordered"'),
 (2,
  '0.019*"place" + 0.015*"restaurant" + 0.014*"like" + 0.012*"toronto" + '
  '0.010*"this" + 0.010*"food" + 0.007*"korean" + 0.006*"best" + '
  '0.005*"restaurants" + 0.005*"better"'),
 (3,
  '0.035*"food" + 0.020*"good" + 0.018*"salad" + 0.016*"service" + '
  '0.012*"place" + 0.011*"restaurant" + 0.009*"menu" + 0.009*"steak" + '
  '0.008*"price" + 0.008*"like"'),
 (4,
  '0.014*"toast" + 0.010*"french" + 0.007*"place" + 0.007*"brunch" + '
  '0.005*"like" + 0.004*"people" + 0.004*"even" + 0.004*"food" + 0.004*"time" '
  '+ 0.004*"room"'),
 (5,
  '0.038*"sushi" + 0.019*"fish" + 0.018*"rolls" + 0.016*"roll" + 0.01

In [159]:
result = lda_model[corpus[0]]
result =sorted(result,key=lambda x:(-x[1],x[0])) 
result[0][0]

11

In [161]:
prob_list = lda_model.get_topic_terms(0)

In [162]:
prob_list = sorted(prob_list,key=lambda x:(-x[1],x[0])) 

In [164]:
prob_list[0:5]

[(527, 0.024300857),
 (1290, 0.023184247),
 (862, 0.016006215),
 (171, 0.015837625),
 (139, 0.015623395)]

In [166]:
val = 3
val1 = 4
val2 = 5

In [174]:
print(val == 4 or val2 == 5 or va3 == 1)

True
